Во многих прикладных задачах приходится иметь дело с гетерографами, т.е. графами, которые содержат информацию о разных типах узлов и разных типах связей.

Проблема: для узлов (ребер) нельзя хранить фичи для всех типов сразу из-за различие в типе данных и размерности. Также новая структура данных требует внесения коррективов в процедуру рассылки сообщений.

## Создание гетерографов

В PyG для описания гетерографов существует структура данных `torch_geometric.data.HeteroData`. 

In [28]:
from torch_geometric.data import HeteroData
import torch as th
# создаем пустой объект
data = HeteroData()
# заполняем фичи узлов
# для типа узлов используем строку
num_nodes = {'paper': 7, 'author': 8, 'institution': 9, 'field_of_study': 10}
num_node_features = {'paper': 5, 'author': 4, 'institution': 3, 'field_of_study': 2}
for node_type in num_nodes:
    data[node_type].x = th.rand(num_nodes[node_type], num_node_features[node_type])
# создаем связи различных типов
# для типа связи используем тройку
num_edges = {('paper', 'cites', 'paper'): 100, 
             ('author', 'writes', 'paper'): 200,
             ('author', 'affiliated_with', 'institution'): 300,
             ('paper', 'has_topic', 'field_of_study'): 400}
num_edge_features = {('paper', 'cites', 'paper'): 5, 
                     ('author', 'writes', 'paper'): 4,
                     ('author', 'affiliated_with', 'institution'): 3,
                     ('paper', 'has_topic', 'field_of_study'): 2} 
for (ut, r, vt) in num_edges:
    data[ut, r, vt].edge_index = th.row_stack((th.randint(low=0, 
                                                          high=num_nodes[ut], 
                                                          size=(1, num_edges[ut, r, vt])),
                                               th.randint(low=0, 
                                                          high=num_nodes[vt], 
                                                          size=(1, num_edges[ut, r, vt]))))
    data[ut, r, vt].edge_attr = th.rand(num_edges[ut, r, vt], num_edge_features[ut, r, vt])                                                     

data

HeteroData(
  paper={ x=[7, 5] },
  author={ x=[8, 4] },
  institution={ x=[9, 3] },
  field_of_study={ x=[10, 2] },
  (paper, cites, paper)={
    edge_index=[2, 100],
    edge_attr=[100, 5]
  },
  (author, writes, paper)={
    edge_index=[2, 200],
    edge_attr=[200, 4]
  },
  (author, affiliated_with, institution)={
    edge_index=[2, 300],
    edge_attr=[300, 3]
  },
  (paper, has_topic, field_of_study)={
    edge_index=[2, 400],
    edge_attr=[400, 2]
  }
)

In [15]:
# у объекта появляются атрибуты типа data.{attribute_name}_dict
data.edge_index_dict

{('paper',
  'cites',
  'paper'): tensor([[1, 0, 4, 2, 4, 2, 5, 4, 0, 0, 2, 6, 1, 1, 4, 2, 4, 5, 1, 1, 4, 5, 6, 0,
          6, 6, 5, 5, 5, 1, 1, 2, 2, 6, 1, 4, 3, 6, 3, 4, 5, 6, 6, 6, 4, 4, 1, 0,
          3, 5, 3, 6, 3, 0, 4, 4, 4, 1, 1, 5, 0, 6, 0, 6, 0, 4, 4, 4, 6, 0, 2, 1,
          5, 2, 3, 1, 3, 4, 2, 1, 5, 4, 2, 4, 1, 1, 2, 6, 5, 4, 5, 5, 2, 1, 1, 0,
          0, 4, 6, 6],
         [4, 6, 3, 5, 0, 3, 5, 6, 6, 2, 2, 0, 2, 5, 5, 5, 2, 4, 3, 0, 1, 3, 4, 1,
          2, 2, 6, 0, 1, 5, 1, 0, 5, 3, 5, 5, 5, 6, 4, 5, 3, 5, 4, 2, 2, 2, 1, 5,
          5, 4, 5, 5, 6, 6, 4, 6, 1, 0, 4, 2, 3, 3, 3, 0, 5, 1, 3, 3, 4, 4, 4, 2,
          6, 4, 6, 3, 4, 0, 2, 5, 6, 2, 2, 3, 2, 2, 2, 5, 4, 1, 1, 1, 2, 6, 3, 0,
          0, 0, 0, 2]]),
 ('author',
  'writes',
  'paper'): tensor([[7, 4, 2, 2, 0, 4, 5, 6, 5, 4, 6, 4, 2, 1, 7, 7, 0, 6, 7, 1, 1, 6, 7, 3,
          0, 7, 1, 2, 5, 2, 6, 0, 0, 1, 6, 6, 0, 1, 7, 7, 2, 0, 4, 7, 2, 7, 4, 0,
          6, 7, 7, 6, 0, 4, 6, 3, 6, 4, 1, 6, 7, 7, 6, 0, 5, 2, 

In [16]:
# можно обращаться к атрибутам узлов по их типу
data['paper']

{'x': tensor([[0.4662, 0.6733, 0.6603, 0.4766, 0.8132],
        [0.5940, 0.1578, 0.8250, 0.3704, 0.9949],
        [0.4961, 0.8509, 0.5635, 0.8434, 0.5183],
        [0.0315, 0.1201, 0.3472, 0.9594, 0.4713],
        [0.7759, 0.0326, 0.7948, 0.0756, 0.9464],
        [0.1572, 0.6340, 0.1196, 0.5474, 0.9780],
        [0.1364, 0.7389, 0.6340, 0.7225, 0.8048]])}

In [11]:
# аналогично с атрибутами ребер
data['author', 'writes', 'paper']

{'edge_index': tensor([[7, 4, 2, 2, 0, 4, 5, 6, 5, 4, 6, 4, 2, 1, 7, 7, 0, 6, 7, 1, 1, 6, 7, 3,
         0, 7, 1, 2, 5, 2, 6, 0, 0, 1, 6, 6, 0, 1, 7, 7, 2, 0, 4, 7, 2, 7, 4, 0,
         6, 7, 7, 6, 0, 4, 6, 3, 6, 4, 1, 6, 7, 7, 6, 0, 5, 2, 2, 3, 5, 5, 5, 5,
         4, 0, 7, 4, 4, 5, 1, 0, 7, 6, 3, 4, 4, 5, 0, 6, 0, 6, 1, 7, 1, 7, 4, 0,
         4, 2, 5, 6, 6, 2, 7, 6, 5, 4, 1, 5, 1, 0, 4, 4, 3, 0, 2, 4, 7, 5, 5, 7,
         6, 7, 1, 2, 1, 2, 0, 0, 3, 0, 0, 0, 5, 3, 5, 6, 6, 1, 5, 6, 0, 4, 6, 3,
         4, 5, 5, 1, 7, 2, 0, 5, 2, 3, 1, 5, 6, 0, 3, 4, 6, 7, 2, 6, 7, 6, 4, 3,
         6, 4, 3, 4, 6, 3, 6, 1, 0, 1, 2, 1, 3, 5, 0, 4, 1, 4, 5, 2, 1, 0, 6, 6,
         2, 7, 7, 4, 0, 4, 5, 4],
        [2, 5, 1, 4, 3, 3, 4, 2, 1, 0, 1, 4, 6, 1, 0, 3, 0, 0, 3, 4, 2, 2, 2, 0,
         4, 0, 1, 3, 0, 4, 2, 2, 2, 4, 0, 6, 6, 3, 2, 1, 3, 2, 5, 0, 5, 4, 3, 1,
         0, 6, 0, 6, 3, 4, 1, 0, 0, 2, 0, 3, 5, 4, 5, 2, 4, 3, 0, 0, 0, 4, 2, 0,
         5, 3, 0, 5, 3, 2, 3, 3, 4, 5, 4, 0, 6, 2, 3, 0, 1, 

In [17]:
# граф можно превратить в однородный, вызвав метод to_homogenious
data_homo = data.to_homogeneous()
data_homo

Data(node_type=[34], edge_index=[2, 1000], edge_type=[1000])

## Преобразования гетерографов

Многие преобразования работают и для гетерографов, но с учетом наличия типов узлов/связей

In [31]:
import torch_geometric.transforms as T

seq_t = T.Compose([T.ToUndirected(),     # добавляет обратные ребра для всех типов ребер
                   T.AddSelfLoops(),     # для всех типов ребер вида ('node_type', 'edge_type', 'node_type') 
                                         # добавляет петли
                   T.NormalizeFeatures() # нормализует все фичи по строкам
                  ])
data_t = seq_t(data)

In [32]:
data_t.metadata()

(['paper', 'author', 'institution', 'field_of_study'],
 [('paper', 'cites', 'paper'),
  ('author', 'writes', 'paper'),
  ('author', 'affiliated_with', 'institution'),
  ('paper', 'has_topic', 'field_of_study'),
  ('paper', 'rev_writes', 'author'),
  ('institution', 'rev_affiliated_with', 'author'),
  ('field_of_study', 'rev_has_topic', 'paper')])